In [1]:
import numpy as np
import pandas as pd
from sqlalchemy import create_engine, text
from sqlalchemy import Table, Column, Integer, String, Float, Date, MetaData
import os
from dotenv import load_dotenv

In [2]:
# Carregar variáveis do arquivo .env
load_dotenv()

# Obter as credenciais do banco de dados do .env
db_host = os.getenv("DB_HOST")
db_port = os.getenv("DB_PORT")
db_name = os.getenv("DB_NAME")
db_user = os.getenv("DB_USER")
db_password = os.getenv("DB_PASSWORD")
excel_path = os.getenv("EXCEL_FILE_PATH")

## Contratos

In [3]:
# Carregar os dados da aba específica do arquivo Excel
df = pd.read_excel(excel_path, sheet_name='contratos_pj') 

# Exibir o número de linhas e colunas
print('Número de linhas e colunas:')
print(df.shape)

print('\n-------------------------------------')

# Exibir os tipos de dados ajustados
print(df.dtypes)

print('\n-------------------------------------')

# Exibir as primeiras linhas dos dados
print('\nVisualização inicial dos dados:')
print(df.head())

Número de linhas e colunas:
(140, 8)

-------------------------------------
EMPRESA                   object
CONSULTAS                  int64
VIDAS ATIVAS               int64
SINISTRALIDADE           float64
PERIODO           datetime64[ns]
MES                       object
ANO                        int64
SETOR                     object
dtype: object

-------------------------------------

Visualização inicial dos dados:
               EMPRESA  CONSULTAS  VIDAS ATIVAS  SINISTRALIDADE    PERIODO  \
0  Saúde Integral Ltda          2            15        0.133333 2024-05-01   
1  Saúde Integral Ltda          5            28        0.178571 2024-04-01   
2  Saúde Integral Ltda          7            26        0.269231 2024-03-01   
3  Saúde Integral Ltda          5            22        0.227273 2024-02-01   
4  Saúde Integral Ltda          6            19        0.315789 2024-01-01   

         MES   ANO    SETOR  
0       MAIO  2024  PUBLICO  
1      ABRIL  2024  PUBLICO  
2      MARÇO  2

In [4]:
# Inserir dados no banco de dados PostgreSQL

# Definir a string de conexão com o banco de dados PostgreSQL
postgres_str = f'postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}'
        
# Configuração da conexão com o banco de dados
engine = create_engine(postgres_str)

# Criação do schema dre_ecbahia
with engine.connect() as connection:
    connection.execute(text("CREATE SCHEMA IF NOT EXISTS telemedicina;"))
    
# from sqlalchemy import Table, Column, Integer, String, Float, MetaData
    
metadata = MetaData(schema='telemedicina')

# Definição da tabela contratos ativos
contratos_ativos = Table('contratos_ativos', metadata,
                         Column('EMPRESA', String),
                         Column('CONSULTAS', Integer),
                         Column('VIDAS_ATIVAS', Integer),
                         Column('SINISTRALIDADE', Float),
                         Column('PERIODO', Date),
                         Column('MES', String),
                         Column('ANO', Integer),
                         Column('SETOR', String),                         
                         )

# Criação da tabela no banco de dados
metadata.create_all(engine)

# Inserção dos dados na tabela f_razao_contabil
df.to_sql('contratos_ativos', engine, schema='telemedicina', if_exists='replace', index=False)

# Consulta para verificar os dados inseridos
with engine.connect() as connection:
    result = connection.execute(text("SELECT * FROM telemedicina.contratos_ativos LIMIT 5;"))
    for row in result:
        print(row)

('Saúde Integral Ltda', 2, 15, 0.13333333333333333, datetime.datetime(2024, 5, 1, 0, 0), 'MAIO', 2024, 'PUBLICO')
('Saúde Integral Ltda', 5, 28, 0.17857142857142858, datetime.datetime(2024, 4, 1, 0, 0), 'ABRIL', 2024, 'PUBLICO')
('Saúde Integral Ltda', 7, 26, 0.2692307692307692, datetime.datetime(2024, 3, 1, 0, 0), 'MARÇO', 2024, 'PUBLICO')
('Saúde Integral Ltda', 5, 22, 0.22727272727272727, datetime.datetime(2024, 2, 1, 0, 0), 'FEVEREIRO', 2024, 'PUBLICO')
('Saúde Integral Ltda', 6, 19, 0.3157894736842105, datetime.datetime(2024, 1, 1, 0, 0), 'JANEIRO', 2024, 'PUBLICO')
